# 전처리 함수화

In [ ]:
import sys
from pathlib import Path

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from code.loading import DataLoader

loader2 = DataLoader(
    start=2020,
    end=2025,
    output_file=DATA_DIR / 'maude_sample.parquet',
    max_workers=4
)

In [ ]:
adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}

df = loader2.load(adapter=adapter, **polars_kwargs)

In [ ]:
import polars as pl

In [ ]:
df.select(pl.count()).collect()

In [ ]:
len(df.columns)

In [ ]:
lf = df.select(['device_0_manufacturer_d_name', 'device_0_manufacturer_d_zip_code', 'device_0_manufacturer_d_postal_code', 'patient_0_patient_sequence_number', 'patient_0_patient_age','patient_0_sequence_number_outcome', 'mdr_text_0_text', 'mdr_text_0_text_type_code'])
lf.head().collect()

In [ ]:
lf.select(pl.col('patient_0_patient_sequence_number').value_counts(sort=True)).collect()

patient_0_patient_sequence_number 버리기.

In [ ]:
lf.select(['device_0_manufacturer_d_zip_code', 'device_0_manufacturer_d_postal_code', 'device_0_manufacturer_d_name']).unique().head().collect()

In [ ]:
null_percentage_lf = lf.select(
    [
        (pl.col(col).null_count() / pl.len()).alias(f"{col}_null_percentage")
        for col in lf.columns
    ]
)

null_percentage_lf.collect()

device_0_manufacturer_d_name, device_0_manufacturer_d_zip_code, device_0_manufacturer_d_postal_code 이 3가지는 null 값이 99.9996%임.

patient_0_patient_sequence_number는 전부 1이라 삭제 예정

patient_0_patient_age는 null값이 2%

patient_0_sequence_number_outcome은 null값이 56.9737%


In [ ]:
null_percentage_df = df.select(
    [
        (pl.col(col).null_count() / pl.len()).alias(f"{col}_null_percentage")
        for col in ['manufacturer_g1_name', 'manufacturer_g1_postal_code', 'manufacturer_g1_zip_code']
    ]
)

null_percentage_df.collect()

manufacturer_g1_name, manufacturer_g1_postal_code로 이름 매칭하기로

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
age = lf.select("patient_0_patient_age").collect().to_pandas()["patient_0_patient_age"]

# Null 제거
age = age.dropna()

plt.hist(age, bins=30)
plt.xlabel("Patient Age")
plt.ylabel("Frequency")
plt.axvline(x=120, color='r', linestyle='--', label='Age 120')
plt.axvline(x=0, color='b', linestyle='--', label='Age 0')
plt.title("Distribution of Patient Age")
plt.show()

In [ ]:
age = lf.select("patient_0_patient_age").collect().to_pandas()["patient_0_patient_age"]
age.head(100)

나이는 120세 이하만 남기기로

In [ ]:
lf.filter(
    pl.col("patient_0_sequence_number_outcome").is_not_null()
).head().collect()


In [ ]:
lf.select(pl.col('patient_0_sequence_number_outcome').value_counts()).collect()

In [ ]:
cleaned = (
    lf
    .with_columns(
        pl.col("patient_0_sequence_number_outcome")
        .str.replace_all(r'^\[|\]$', "")     # 대괄호 제거
        .str.replace_all(r"'", "")  # 따옴표 제거
        .str.split(",")    # 리스트로 변환
        .alias("out_list")
    )
)

result = (
    cleaned
    .explode("out_list")
    .select(
        pl.col("out_list").str.strip_chars().alias("outcome")  # explode 후 strip
    )
    .group_by("outcome")
    .count()
    .sort("count", descending=True)
    .collect().to_pandas()
)
result

Death - D

Disability - S

일 가능성이 높아보임. 그렇게 처리할 예정.

1. udi id로 검색하기 https://api.fda.gov/device/udi.json?search=identifiers.id:(01)SW11677
2. 안 나오면 model 번호로 검색하기 https://api.fda.gov/device/udi.json?search=version_or_model_number:SW11677

In [ ]:
df.select('device_0_udi_di').head().collect()

In [ ]:
df.select(pl.col("device_0_device_operator").unique()).collect()

In [ ]:
df.select([
    pl.col("device_0_device_operator").n_unique().alias("n_unique"),
    pl.col("device_0_device_operator").null_count().alias("null_count")
]).collect()

In [ ]:
df.head(10).collect()

# 해야 하는 거
1. patient_0_patient_age 120 이하 자르기
2. patient_0_sequence_number_outcome one-hot encoding
3. manufacturer_g1_name postal code나 zip code 사용해서 매칭
4. device_operator other 결측치 처리, 타입 bool 변환
5. device_report_product_code 특문 시작 코드 2개 삭제(-, ---)
6. openfda_device_name 
    * 기본적인 전처리만 하고 제품 코드랑 비교해서 통일시키기 -> device_name이 product_code와 1대1이 될 수 있게 만들기

## 1. age 120 이하 자르기

In [ ]:
def age_cut(lf, col_name):
    """
    age col이 들어오면 0~120으로 범위를 한정해주는 함수.
    0 이하는 0으로, 120 이상은 120으로 limit
    """
    return lf.with_columns(
        pl.col(col_name)
        .str.replace("NA", "")  # "NA"를 빈 문자열로 (null 처리)
        .str.extract(r"(\d+)", 1)  # 숫자만 추출
        .cast(pl.Float64)  # 숫자로 변환
        .clip(0, 120)  # 0~120 범위로 제한
        .alias(col_name)
    )

In [ ]:
age_limited = age_cut(df, "patient_0_patient_age")
age_limited.select(
    pl.col("patient_0_patient_age").min().alias("min_age"),
    pl.col("patient_0_patient_age").max().alias("max_age")
).collect()

In [ ]:
age = (
    age_limited
    .select("patient_0_patient_age")
    .filter(pl.col("patient_0_patient_age").is_not_null())
    .collect()
    .sample(1000000)
    .to_pandas()["patient_0_patient_age"]
)

plt.hist(age, bins=30)
plt.xlabel("Patient Age")
plt.ylabel("Frequency")
plt.legend()
plt.title("Distribution of Patient Age")
plt.show()

# 2. sequence_number_outcome one-hot encoding

In [ ]:
def sequence_number_outcome_clean(lf, col_name):
    """
    patient_0_sequence_number_outcome 컬럼을 one hot encoding (LazyFrame 유지)
    """
    
    outcome_mapping = {
        'Life Threatening': 'L',
        'Hospitalization': 'H',
        'Disability': 'S',
        'Congenital Anomaly': 'C',
        'Required Intervention': 'R',
        'Death': 'D',
        'Other': 'O',
        'Invalid Data': 'O',
        'Unknown': 'O',
        'No Information': 'O',
        'Not Applicable': 'O',
    }
    
    # 모든 가능한 outcome 코드
    all_outcomes = ['L', 'H', 'S', 'C', 'R', 'D', 'O']
    
    result = lf.with_columns(
        pl.col(col_name)
        .str.replace_all(r'^\[|\]$', "")
        .str.replace_all(r"'", "")
        .str.split(",")
        .list.eval(pl.element().str.strip_chars())
        .alias("_outcome_list")
    )
    
    # 매핑 적용
    for key, value in outcome_mapping.items():
        result = result.with_columns(
            pl.col("_outcome_list")
            .list.eval(pl.element().str.replace(key, value))
            .alias("_outcome_list")
        )
    
    # 각 outcome에 대해 one-hot 컬럼 생성
    for outcome in all_outcomes:
        result = result.with_columns(
            pl.col("_outcome_list")
            .list.contains(outcome)
            .cast(pl.Int32)
            .alias(f"outcome_{outcome}")
        )
    
    result = result.drop("_outcome_list", col_name)
    
    return result

In [ ]:
# 상위 10개 행만 처리해서 확인
test_result = (
    sequence_number_outcome_clean(lf.head(100), "patient_0_sequence_number_outcome")
    .collect()
)

test_result.select(['outcome_L', 'outcome_H', 'outcome_S', 'outcome_C', 'outcome_R', 'outcome_D', 'outcome_O'])

# 3. manufacturer_g1_name postal code나 zip code 사용해서 매칭

In [ ]:
def manufacturer_postal_match(lf, name_col, postal_col):
    """
    manufacturer_g1_name을 manufacturer_g1_postal_code로 매칭시켜서
    이름 없는 것들 채우고 있는 것들 통일하기
    """
    # 우편번호별로 가장 빈도가 높은 이름 선택
    postal_to_name = (
        lf
        .select([name_col, postal_col])
        .filter(pl.col(name_col).is_not_null() & pl.col(postal_col).is_not_null())
        .unique()
        .group_by(postal_col)
        .agg(
            pl.col(name_col).mode().first().alias("canonical_name")
        )
    )
    
    # 원본 데이터에 매핑 테이블 조인
    result = (
        lf
        .join(
            postal_to_name,
            on=postal_col,
            how="left"
        )
        .with_columns(
            # canonical_name이 있으면 사용, 없으면 원래 이름 유지
            pl.coalesce(pl.col("canonical_name"), pl.col(name_col)).alias(name_col)
        )
        .drop("canonical_name")
    )
    
    return result

In [ ]:
match_test_result = manufacturer_postal_match(lf.head(100000), 'device_0_manufacturer_d_name', 'device_0_manufacturer_d_postal_code').collect()

match_test_result.select(['device_0_manufacturer_d_name', 'device_0_manufacturer_d_postal_code'])

# 4. device_operator other 결측치 처리, 타입 bool 변환

In [ ]:
def device_operator_clean(lf, col_name):
    """
    device_0_device_operator 컬럼 클린징 함수
    "HEALTH PROFESSIONAL"을 1, "LAY USER/PATIENT"을 0, 그 외는 null로 변환
    """
    return lf.with_columns(
        pl.when(pl.col(col_name) == "HEALTH PROFESSIONAL")
        .then(True)
        .when(pl.col(col_name) == "LAY USER/PATIENT")
        .then(False)
        .otherwise(None)
        .cast(pl.Boolean)
        .alias(col_name)
    )

# 5. device_report_product_code 특문 시작 코드 2개 삭제(-, ---)

In [ ]:
def product_code_clean(lf, col_name):
    """
    device_0_device_product_code 컬럼의 영어 대문자 이외 문자 제거
    """
    return lf.with_columns(
        pl.col(col_name)
        .str.replace_all(r'[^A-Z]', '')  # 영어 대문자 이외 문자 제거
        .alias(col_name)
    )

# 6. openfda_device_name 
* 기본적인 전처리만 하고 제품 코드랑 비교해서 통일시키기 -> device_name이 product_code와 1대1이 될 수 있게 만들기

In [ ]:
def device_name_clean(lf, col_name):
    """
    device_0_device_name 컬럼의 특수문자 제거 및 소문자 변환
    """
    return lf.with_columns(
        pl.col(col_name)
        .str.replace_all(r'[^a-zA-Z0-9\s]', '')  # 특수문자 제거
        .str.to_lowercase()  # 소문자 변환
        .str.strip_chars()  # 앞뒤 공백 제거
        .str.replace_all(r'\s+', ' ')  # 연속된 공백을 하나로
        .alias(col_name)
    )
    
def device_name_product_code_match(lf, name_col, code_col):
    """
    device_0_device_name과 device_0_device_product_code를 매칭시켜서
    한 product_code에 여러 device_name이 있는 경우 가장 빈도가 높은 이름으로 통일
    """
    # product_code별로 가장 빈도가 높은 device_name 선택
    code_to_name = (
        lf
        .select([name_col, code_col])
        .filter(pl.col(name_col).is_not_null() & pl.col(code_col).is_not_null())
        .group_by([code_col, name_col])  # 먼저 그룹화해서 빈도 계산
        .count()
        .sort("count", descending=True)
        .group_by(code_col)
        .first()  # 각 code별 가장 빈도 높은 이름 선택
        .select([code_col, pl.col(name_col).alias("canonical_name")])
    )
    
    # 원본 데이터에 매핑 테이블 조인
    result = (
        lf
        .join(
            code_to_name,
            on=code_col,
            how="left"
        )
        .with_columns(
            pl.coalesce(pl.col("canonical_name"), pl.col(name_col)).alias(name_col)
        )
        .drop("canonical_name")
    )
    
    return result

In [ ]:
# 테스트
# 1. device_name 클린징
cleaned = device_name_clean(df.head(10000), "device_0_openfda_device_name")

# 2. product_code와 매칭
result = device_name_product_code_match(cleaned, "device_0_openfda_device_name", "device_0_device_report_product_code")

# 확인
result.select(['device_0_openfda_device_name','device_0_device_report_product_code']).collect()